# Slow Analog Inputs

Slow analog inputs provide a convenient way to acquire slow-changing analog signals.
All four analog inputs are located on the E2 connector, depicted in the picture below, and can measure voltages between 0 V and 3.5 V.

![alt text](../img/Extension_connector.png "STEMlab extention connector.")

## Libraries and FPGA image

We will start by importing the *rp* (Red Pitaya) and *rp_overlay* libraries, loading the *v0.94* FPGA image, and initializing the Red Pitaya.

In [ ]:
from rp_overlay import overlay
import rp

fpga = overlay()
rp.rp_Init()

## Macros
Throughout this tutorial we will mention macros multiple times. Here is a complete list of analog macros that will come in handy when customising this notebook. The marcos are a part of the **rp** library.

- **Analog outputs** - RP_AOUT0, RP_AOUT1, ..., RP_AOUT3
- **Analog inputs** - RP_AIN0, RP_AIN1, ..., RP_AIN3

## Controling Analog Inputs
The analog inputs can be read in two different ways; either get the value in Volts or get a RAW value directly from the ADC. Analog input numbers range from 0 to 3.

### Reading value in Volts
- **rp_AIpinGetValue(analog_input_number)**
- **rp_ApinGetValue(analog_input_macro)**

### Reading RAW value
- **rp_AIpinGetValueRaw(analog_input_number)**
- **rp_ApinGetValueRaw(analog_input_macro)**

### Reading analog input range
To get the analog input range use the following function:
- **rp_ApinGetRange(analog_pin_macro)**

### Reseting analog pins
To reset all analog pins to their default state (output 0 V) use:
- **rp_ApinReset()**


First we reset all the analog pins.

In [ ]:
rp.rp_ApinReset()

Next, we will read the value on the slow **Analog input 0** and print the value using both options:

In [ ]:
print(f"Analog input 0: {rp.rp_AIpinGetValue(0)[1]} V")               # Specifying the pin value (0, 1, 2, or 3)
print(f"Analog input 0: {rp.rp_ApinGetValue(rp.RP_AIN0)[1]} V")       # Specifying the pin with a Macro

You also have an option to get the RAW value form the internal ADC:

In [ ]:
print(f"Analog input 0 RAW value: {rp.rp_AIpinGetValueRaw(0)[1]}")
print(f"Analog input 0 RAW value: {rp.rp_ApinGetValueRaw(rp.RP_AIN0)[1]}")

Using the Python f-strings, we can also format the output, so that only 2 decimal places are displayed.

In [ ]:
print(f"Analog input 0 value: {rp.rp_ApinGetValue(rp.RP_AIN0)[1]:.2f}")

## Reading all 4 Analog inputs

Here, we use a *for* loop to read the values from all four channels with the same formatting as in the previous example.

In [ ]:
for i in range(0,4):
    print(f"Analog input {i}: {rp.rp_AIpinGetValue(i)[1]:.2f} V")

Finaly, at the end of the program, we should always release the used resources. Execute the following cell.

In [ ]:
rp.rp_Release()

## Logging and plotting live data

In this example, we will measure all four analog inputs every 0.5 s. The data will be stored in a list `y` and visualized with BokehJS.

### Generate data

For data generation, we will use Jupyter's ability to run more than one notebook simultaneously. However, before we do that, we will have to manually connect the output pins to the input pins, as depicted in the picture below.

![analog Loop Back](img/analogLoopBack.png "STEMlab slow analog loop back.")


Now that we have created a physical loopback, open the [analog signal generator](analog_signal_generator.ipynb) notebook, run all the cells, and then return to this notebook. This will start an infinite loop that generates sine signals on all four analog outputs with different amplitudes.

**Note:**  
We do not recommend running more than four notebooks simultaneously due to limited resources on STEMlab.

Since we want to take measurements every 0.01 seconds, we need to import a time module and, to be able to plot data, import a few modules from Bokeh.

In [ ]:
import time
import numpy as np
import rp

from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool, Range1d
from bokeh.plotting import figure
from bokeh.resources import INLINE
output_notebook(resources=INLINE)

rp.rp_Init()

Next, we will prepare the parameters of the plot, such as line colours, axis labels, etc.

We will also prepare an empty plot to populate with the next cell.

In [ ]:
colors = ('red', 'blue', 'green', 'orange')
hover = HoverTool(mode= 'vline', tooltips= [("T", "@x"), ("V", "@y")])
tools = "pan,wheel_zoom,box_zoom,reset,crosshair"
p = figure(height= 400, width= 900, title= "XADC log", toolbar_location= "above", tools= (tools, hover))
p.xaxis.axis_label='time [s]'
p.y_range = Range1d(0, 2)
p.yaxis.axis_label = 'voltage [V]'

T = 0.01        # sampling period
STEPS = 60
INIT_VAL = np.zeros(STEPS)
x = np.linspace(0, STEPS * T, num= STEPS)
y = [np.full(STEPS, INIT_VAL, dtype= float), np.full(STEPS, INIT_VAL, dtype= float), np.full(STEPS, INIT_VAL, dtype= float), np.full(STEPS, INIT_VAL, dtype= float)]


r = [p.line(x, y[ch], line_width=1, line_alpha=0.7, color=colors[ch], legend_label=f"AI {ch}") for ch in range(0,4)]


# get an explicit handle to update the next show cell with
target = show(p, notebook_handle=True)

Here, we will start sampling Analog inputs while also populating the plot. While the sampling rate is set to 2 samples per second, the actual sampling rate is lower as we use the sleep function to delay the sampling by 2 seconds. However, we update the plot in the same loop, which takes some time since the sampling rate is lower.

In [ ]:
for i in range(STEPS):
    for ch in range(0,4):
        y = r[ch].data_source.data['y']
        y[i] = rp.rp_AIpinGetValue(ch)[1]
        r[ch].data_source.data['y'] = y 
    
    # Push updates to the plot continuously using the handle (interrupt the notebook kernel to stop)
    push_notebook(handle=target)
    time.sleep(T)

Releasing resources:

In [ ]:
rp.rp_Release()